In [ ]:
#| export
import polars as pl
import pyarrow.parquet as pq
import pyarrow.dataset as ds
from s3fs import S3FileSystem
import pandas as pd
import plotly.express as px
from fastcore.basics import patch
import re

In [ ]:
path = "./data/v1/logs/*/*/*/*.parquet"
logs = pl.scan_parquet(path)
logs.collect()

In [ ]:
(
    pl.scan_parquet(path)
        .with_columns([
            pl.col("body").str.json_path_match("$.type").alias("type"),
            pl.col("body").str.json_path_match("$.action").alias("action"),
            pl.col("body").str.json_path_match("$.actor_description").alias("username"),
        ])
        .filter(
            (pl.col("service") == "connect") &
            (pl.col("type") == "audit") &
            (pl.col("action") == "add_group")
        )
        # .with_columns(
        #     pl.col("attributes").arr.to_struct()
        # )
        # .select([
        #     pl.col("attributes").struct.unnest()
        # ])
    .head(1)
    .explode("attributes")
    # .sort("attribtutes")
    .collect()
)

In [ ]:

def get_connect_logins(path):
    return (
        pl.scan_parquet(path)
        .with_columns([
            pl.col("body").str.json_path_match("$.type").alias("type"),
            pl.col("body").str.json_path_match("$.action").alias("action"),
            pl.col("body").str.json_path_match("$.actor_description").alias("username"),
        ])
        .filter(
            (pl.col("service") == "connect") &
            (pl.col("type") == "audit") &
            (pl.col("action") == "user_login")
        )
        .select("host", "timestamp", "username", "action", "type")
        .collect()
    )

path = "./data/v1/logs/*/*/*/*.parquet"
get_connect_logins(path)

In [ ]:
def get_workbench_logins(path):
    return (
        pl.scan_parquet(path)
        .with_columns([
            pl.col("body").str.json_path_match("$.type").alias("type"),
            pl.col("body").str.json_path_match("$.action").alias("action"),
            pl.col("body").str.json_path_match("$.username").alias("username"),
        ])
        .filter(
            (pl.col("service") == "workbench") &
            (pl.col("type") == "auth_login")
        )
        .select("host", "timestamp", "username", "action", "type")
        .collect()
    )

get_workbench_logins(path)

host,timestamp,username,action,type
str,datetime[ms],str,str,str
"""rstudio-workbe…",2023-04-04 16:21:48.361,"""monanshi.shah""",null,"""auth_login"""
"""rstudio-workbe…",2023-04-06 21:44:28.264,"""lisa.anders""",null,"""auth_login"""
"""rstudio-workbe…",2023-04-07 03:46:57.960,"""james""",null,"""auth_login"""
"""rstudio-workbe…",2023-04-11 17:14:54.265,"""lisa.anders""",null,"""auth_login"""
"""rstudio-workbe…",2023-05-05 13:47:58.865,"""cole""",null,"""auth_login"""
"""rstudio-workbe…",2023-05-08 15:19:22.190,"""andrie""",null,"""auth_login"""
"""rstudio-workbe…",2023-05-08 16:58:39.698,"""monanshi.shah""",null,"""auth_login"""
"""rstudio-workbe…",2023-05-09 14:30:10.252,"""monanshi.shah""",null,"""auth_login"""


In [ ]:

scan_chronicle_logs("./data", "2023/04/03").logs.filter_type("username")
logs = scan_chronicle_logs("./data", "2023/04/03")
(
    logs
    .with_columns([
        pl.col("body").str.json_path_match("$.type").alias(".type")
    ])
    .unique(".type")
    .collect()
)